## Project code
#### Using PuLP,  a python library for optimizaiton probelms

- Identify the Decision Variables
- Formulate the Objective Function using the decision variables, maximize total score
- Formulate the Constraints, in terms of the decision variables.
- Identify the Data needed for the objective function and constraints


### Import Libraries

In [1]:
import urllib, json
import pandas as pd
import re
from itertools import permutations
from pulp import *

### Reading th Fantasy Football Salary and Porjection

In [2]:
dataset = pd.read_csv("fantasy_football_salary_and_projection_w1.csv", header = 0)
print(dataset.shape)
dataset.head(10)

(300, 10)


,Rank,Name,Team,Position,Week,Opponent,UpcomingOpponentRank,UpcomingOpponentPositionRank,FanDuelSalary,FantasyPointsFanDuel
0,83,Baltimore Ravens,BAL,DST,1,BUF,NaN,NaN,4800,10.3
1,100,Jacksonville Jaguars,JAX,DST,1,NYG,NaN,NaN,4700,9.7
2,110,New Orleans Saints,NO,DST,1,TB,NaN,NaN,4600,9.1
3,123,Detroit Lions,DET,DST,1,NYJ,NaN,NaN,4700,8.5
4,129,Pittsburgh Steelers,PIT,DST,1,CLE,NaN,NaN,4200,8.4
5,145,Tennessee Titans,TEN,DST,1,MIA,NaN,NaN,4200,8.1
6,145,New England Patriots,NE,DST,1,HOU,NaN,NaN,4200,8.1
7,157,Washington Redskins,WAS,DST,1,ARI,NaN,NaN,4000,8.0
8,161,Green Bay Packers,GB,DST,1,CHI,NaN,NaN,4100,7.9
9,164,Minnesota Vikings,MIN,DST,1,SF,NaN,NaN,4500,7.8


### Clean Data
#### Drop columns that aren't used

In [3]:
dataset = dataset.drop(['Team', 'Week', 'Opponent', 'UpcomingOpponentRank', 'UpcomingOpponentPositionRank'], axis = 1)
dataset.head(10)

,Rank,Name,Position,FanDuelSalary,FantasyPointsFanDuel
0,83,Baltimore Ravens,DST,4800,10.3
1,100,Jacksonville Jaguars,DST,4700,9.7
2,110,New Orleans Saints,DST,4600,9.1
3,123,Detroit Lions,DST,4700,8.5
4,129,Pittsburgh Steelers,DST,4200,8.4
5,145,Tennessee Titans,DST,4200,8.1
6,145,New England Patriots,DST,4200,8.1
7,157,Washington Redskins,DST,4000,8.0
8,161,Green Bay Packers,DST,4100,7.9
9,164,Minnesota Vikings,DST,4500,7.8


#### Add FLEX position tag to eligible players (RB, WR, TE)

In [4]:
flex = dataset[dataset.Position.isin(["RB","WR","TE"])].copy()
flex.Position = "FLEX"
dataset = pd.concat([dataset, flex])
dataset = dataset.drop('Rank', axis=1)

dataset.head()

,Name,Position,FanDuelSalary,FantasyPointsFanDuel
0,Baltimore Ravens,DST,4800,10.3
1,Jacksonville Jaguars,DST,4700,9.7
2,New Orleans Saints,DST,4600,9.1
3,Detroit Lions,DST,4700,8.5
4,Pittsburgh Steelers,DST,4200,8.4


#### Attach salaries and scores on position

In [5]:
dataset[dataset.Position=="QB"].head(15)

salaries = {}
points = {}
for pos in dataset.Position.unique():
    available_pos = dataset[dataset.Position == pos]
    salary = list(available_pos[["Name","FanDuelSalary"]].set_index("Name").to_dict().values())[0]
    point = list(available_pos[["Name","FantasyPointsFanDuel"]].set_index("Name").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 2,
    "TE": 1,
    "DST": 1,
    "K": 1,
    "FLEX": 1
}


### Add Constraints and Setup Problem
#### Salaries and Positional

In [6]:
SALARY_CAP = 50000
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [7]:
prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
position_constraints = []

# Setting up the rewards, costs, and positional constraints
for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) == pos_num_available[k]
    
# Setting up constraint to not pick the same player for 2 positions   
for position_name in ('WR', 'RB', 'TE'):
    for player in _vars[position_name]:
        prob += lpSum([_vars[p][player] for p in ['FLEX', position_name]]) <= 1
    
prob += lpSum(rewards)
prob += lpSum(costs) <= SALARY_CAP

print(prob)

Fantasy:
MAXIMIZE
7.3*DST_Arizona_Cardinals + 6.0*DST_Atlanta_Falcons + 10.3*DST_Baltimore_Ravens + 5.7*DST_Buffalo_Bills + 7.7*DST_Carolina_Panthers + 6.8*DST_Chicago_Bears + 7.5*DST_Cincinnati_Bengals + 4.6*DST_Cleveland_Browns + 6.1*DST_Dallas_Cowboys + 7.8*DST_Denver_Broncos + 8.5*DST_Detroit_Lions + 7.9*DST_Green_Bay_Packers + 4.8*DST_Houston_Texans + 4.9*DST_Indianapolis_Colts + 9.7*DST_Jacksonville_Jaguars + 5.8*DST_Kansas_City_Chiefs + 6.9*DST_Los_Angeles_Chargers + 7.8*DST_Los_Angeles_Rams + 6.2*DST_Miami_Dolphins + 7.8*DST_Minnesota_Vikings + 8.1*DST_New_England_Patriots + 9.1*DST_New_Orleans_Saints + 5.7*DST_New_York_Giants + 6.1*DST_New_York_Jets + 5.2*DST_Oakland_Raiders + 6.1*DST_Philadelphia_Eagles + 8.4*DST_Pittsburgh_Steelers + 5.1*DST_San_Francisco_49ers + 7.5*DST_Seattle_Seahawks + 4.2*DST_Tampa_Bay_Buccaneers + 8.1*DST_Tennessee_Titans + 8.0*DST_Washington_Redskins + 14.26*FLEX_AJ_Green + 4.88*FLEX_Adam_Humphries + 12.28*FLEX_Adam_Thielen + 7.36*FLEX_Adrian_Peterson

### Solve Optimization Problem

In [8]:
prob.solve()

1

In [9]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [10]:
summary(prob)

Variables:

DST_Miami_Dolphins = 1.0
FLEX_James_Conner = 1.0
K_Randy_Bullock = 1.0
QB_Ryan_Fitzpatrick = 1.0
RB_Alvin_Kamara = 1.0
RB_David_Johnson = 1.0
TE_Dallas_Goedert = 1.0
WR_Keelan_Cole = 1.0
WR_Nelson_Agholor = 1.0
---------------------------------------

Constraints:
3200*1.0 + 5000*1.0 + 3900*1.0 + 6200*1.0 + 8700*1.0 + 8600*1.0 + 4000*1.0 + 4500*1.0 + 5900*1.0 = 50000.0
---------------------------------------

Score:
6.2*1.0 + 12.76*1.0 + 7.89*1.0 + 15.58*1.0 + 19.72*1.0 + 19.52*1.0 + 4.57*1.0 + 8.85*1.0 + 10.6*1.0 = 105.69


### Week 1

Variables:

DST_Miami_Dolphins = 1.0 <br>
FLEX_James_Conner = 1.0 <br>
K_Randy_Bullock = 1.0 <br>
QB_Ryan_Fitzpatrick = 1.0 <br>
RB_Alvin_Kamara = 1.0 <br>
RB_David_Johnson = 1.0 <br>
TE_Dallas_Goedert = 1.0 <br>
WR_Keelan_Cole = 1.0 <br>
WR_Nelson_Agholor = 1.0 

**---------------------------------------**

Constraints:
3200* 1.0 + 5000* 1.0 + 3900* 1.0 + 6200* 1.0 + 8700* 1.0 + 8600* 1.0 + 4000* 1.0 + 4500* 1.0 + 5900* 1.0 = 50000.0

**---------------------------------------**

Score:
6.2* 1.0 + 12.76* 1.0 + 7.89* 1.0 + 15.58* 1.0 + 19.72* 1.0 + 19.52* 1.0 + 4.57* 1.0 + 8.85* 1.0 + 10.6* 1.0 = 105.69


### Week 10

Variables:

DST_New_Orleans_Saints = 1.0 <br>
FLEX_Isaiah_Crowell = 1.0 <br>
K_Dustin_Hopkins = 1.0 <br>
QB_Ben_Roethlisberger = 1.0 <br>
RB_Dion_Lewis = 1.0 <br>
RB_James_Conner = 1.0 <br>
TE_Jordan_Reed = 1.0 <br>
WR_Josh_Doctson = 1.0 <br>
WR_Marquez_Valdes_Scantling = 1.0 <br>

**---------------------------------------**

Constraints: 
3200* 1.0 + 5400* 1.0 + 4100* 1.0 + 7800* 1.0 + 5800* 1.0 + 8200* 1.0 + 5200* 1.0 + 4900* 1.0 + 5400* 1.0 = 50000.0

**---------------------------------------**

Score:
6.6* 1.0 + 10.11* 1.0 + 7.99* 1.0 + 20.49* 1.0 + 11.82* 1.0 + 18.71* 1.0 + 8.52* 1.0 + 8.31* 1.0 + 10.56* 1.0 = 103.11

### Week 11

Variables:

DST_Seattle_Seahawks = 1.0 <br>
FLEX_John_Brown = 1.0 <br>
K_Wil_Lutz = 1.0 <br>
QB_Carson_Wentz = 1.0 <br>
RB_David_Johnson = 1.0 <br>
RB_Dion_Lewis = 1.0 <br>
TE_James_O'Shaughnessy = 1.0 <br>
WR_Devin_Funchess = 1.0 <br>
WR_Marquez_Valdes_Scantling = 1.0 <br>

**---------------------------------------**

Constraints:
3100* 1.0 + 5400* 1.0 + 5000* 1.0 + 7700* 1.0 + 7900* 1.0 + 5900* 1.0 + 4000* 1.0 + 5700* 1.0 + 5300* 1.0 = 50000.0

**---------------------------------------**

Score:
6.7* 1.0 + 10.19* 1.0 + 11.28* 1.0 + 21.25* 1.0 + 18.56* 1.0 + 12.16* 1.0 + 5.43* 1.0 + 10.95* 1.0 + 10.88* 1.0 = 107.39999999999999

### Week 12

Variables:

DST_Cleveland_Browns = 1.0 <br>
FLEX_James_Conner = 1.0 <br>
K_Wil_Lutz = 1.0 <br>
QB_Nick_Mullens = 1.0 <br>
RB_Gus_Edwards = 1.0 <br>
RB_Matt_Breida = 1.0 <br>
TE_Cameron_Brate = 1.0 <br>
WR_D.J._Moore = 1.0 <br>
WR_Jarvis_Landry = 1.0 <br>

**---------------------------------------**

Constraints:
3100* 1.0 + 7900* 1.0 + 5000* 1.0 + 6000* 1.0 + 5300* 1.0 + 6400* 1.0 + 4400* 1.0 + 5800* 1.0 + 6100* 1.0 = 50000.0

**---------------------------------------**

Score:
6.6* 1.0 + 18.39* 1.0 + 11.51* 1.0 + 16.69* 1.0 + 10.23* 1.0 + 13.51* 1.0 + 8.03* 1.0 + 10.5* 1.0 + 11.15* 1.0 = 106.61000000000001


### Week 13

Variables:

DST_Carolina_Panthers = 1.0 <br>
FLEX_TJ_Yeldon = 1.0 <br>
K_Michael_Badgley = 1.0 <br>
QB_Jameis_Winston = 1.0 <br>
RB_Austin_Ekeler = 1.0 <br>
RB_Spencer_Ware = 1.0 <br>
TE_Eric_Ebron = 1.0 <br>
WR_Corey_Davis = 1.0 <br>
WR_Tyreek_Hill = 1.0 <br>

**---------------------------------------**

Constraints:
3800* 1.0 + 4800* 1.0 + 3400*  1.0 + 7500* 1.0 + 5400* 1.0 + 5200* 1.0 + 5600* 1.0 + 5900* 1.0 + 8400* 1.0 = 50000.0

**---------------------------------------**

Score:
8.5* 1.0 + 10.1* 1.0 + 8.34* 1.0 + 21.31* 1.0 + 13.54* 1.0 + 14.33* 1.0 + 11.13* 1.0 + 11.64* 1.0 + 17.18* 1.0 = 116.07

### Week 14

Variables:

DST_Dallas_Cowboys = 1.0 <br>
FLEX_Justin_Jackson = 1.0 <br>
K_Steven_Hauschka = 1.0 <br>
QB_Jameis_Winston = 1.0 <br>
RB_Alvin_Kamara = 1.0 <br>
RB_Jaylen_Samuels = 1.0 <br>
TE_Eric_Ebron = 1.0 <br>
WR_Chris_Godwin = 1.0 <br>
WR_Sterling_Shepard = 1.0 <br>

**---------------------------------------**

Constraints:
3700* 1.0 + 4500* 1.0 + 4000* 1.0 + 7700* 1.0 + 8300* 1.0 + 4600* 1.0 + 6400* 1.0 + 5600* 1.0 + 5200* 1.0 = 50000.0

**---------------------------------------**

Score:
7.8* 1.0 + 10.0* 1.0 + 8.21* 1.0 + 21.2* 1.0 + 19.02* 1.0 + 12.14* 1.0 + 12.11* 1.0 + 11.03* 1.0 + 10.6* 1.0 = 112.11


### Week 15

Variables:

DST_Chicago_Bears = 1.0 <br>
FLEX_Damien_Williams = 1.0<br>
K_Greg_Zuerlein = 1.0<br>
QB_Nick_Foles = 1.0<br>
RB_Jaylen_Samuels = 1.0<br>
RB_Justin_Jackson = 1.0<br>
TE_Travis_Kelce = 1.0<br>
WR_Amari_Cooper = 1.0<br>
WR_Sterling_Shepard = 1.0<br>

**---------------------------------------**

Constraints:
3700* 1.0 + 5500* 1.0 + 5000* 1.0 + 6000* 1.0 + 5500* 1.0 + 4500* 1.0 + 7900* 1.0 + 6600* 1.0 + 5200* 1.0 = 49900.0

**---------------------------------------**

Score:
9.0* 1.0 + 11.57* 1.0 + 11.43* 1.0 + 16.53* 1.0 + 12.38* 1.0 + 13.67* 1.0 + 15.56* 1.0 + 14.23* 1.0 + 10.03* 1.0 = 114.4

### Week 16

Variables:

DST_Philadelphia_Eagles = 1.0<br>
FLEX_Elijah_McGuire = 1.0<br>
K_Brandon_McManus = 1.0<br>
QB_Nick_Foles = 1.0<br>
RB_Ezekiel_Elliott = 1.0<br>
RB_Jamaal_Williams = 1.0<br>
TE_Eric_Ebron = 1.0<br>
WR_Dante_Pettis = 1.0<br>
WR_Sterling_Shepard = 1.0<br>

**---------------------------------------**

Constraints:
3200* 1.0 + 5700* 1.0 + 4000* 1.0 + 6000* 1.0 + 8800* 1.0 + 5800* 1.0 + 6100* 1.0 + 5400* 1.0 + 5000* 1.0 = 50000.0

**---------------------------------------**

Score:
7.1* 1.0 + 12.39* 1.0 + 8.14* 1.0 + 16.71* 1.0 + 20.99* 1.0 + 13.04* 1.0 + 10.87* 1.0 + 10.0* 1.0 + 9.18* 1.0 = 108.42000000000002


### Week 17

Variables:

DST_New_York_Giants = 1.0 <br>
FLEX_Dwayne_Washington = 1.0<br>
K_Harrison_Butker = 1.0<br>
QB_Patrick_Mahomes = 1.0<br>
RB_Chris_Carson = 1.0<br>
RB_Royce_Freeman = 1.0<br>
TE_Eric_Ebron = 1.0<br>
WR_Jordy_Nelson = 1.0<br>
WR_Sterling_Shepard = 1.0<br>

**---------------------------------------**

Constraints:
3300* 1.0 + 4500* 1.0 + 5000* 1.0 + 9500* 1.0 + 6400* 1.0 + 4700* 1.0 + 5800* 1.0 + 5200* 1.0 + 5500* 1.0 = 49900.0

**---------------------------------------**

Score:
7.6* 1.0 + 9.34* 1.0 + 10.66* 1.0 + 23.7* 1.0 + 14.0* 1.0 + 10.15* 1.0 + 10.61* 1.0 + 10.19* 1.0 + 9.89* 1.0 = 106.14